In [2]:
from verifiers import GenerateOutputs

In [7]:
from typing import Any
from pydantic import field_serializer, BaseModel

class Trajectories(GenerateOutputs):

    @field_serializer("*", mode="plain")
    def serialize_field(self, value: Any) -> object:
        if isinstance(value, BaseModel):
            return value.model_dump()
        elif isinstance(value, list):
            return [self.serialize_field(v) for v in value]
        elif isinstance(value, dict):
            return {k: self.serialize_field(v) for k, v in value.items()}
        else:
            return value
    

In [8]:
import pickle

RESULTS_PATH = "/workspace/exploration-hacking/artifacts/data/dfalck/science_locking/post_sft_eval_train.pkl"

with open(RESULTS_PATH, "rb") as f:
    results = pickle.load(f)

In [9]:
trajectories = Trajectories(**{k: getattr(results, k) for k in type(results).model_fields})

In [16]:
import json

d = trajectories.model_dump()
string = json.dumps(d, indent=4)
with open("trajectories.json", "w") as f:
    f.write(string)

In [17]:
trajectories.model_fields

/tmp/ipykernel_18343/1289325718.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  trajectories.model_fields


{'prompt': FieldInfo(annotation=list[Union[str, list[Annotated[Union[ChatCompletionDeveloperMessageParam, ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam, ChatCompletionAssistantMessageParam, ChatCompletionToolMessageParam, ChatCompletionFunctionMessageParam], SkipValidation]]]], required=True),
 'completion': FieldInfo(annotation=list[Union[str, list[Annotated[Union[ChatCompletionDeveloperMessageParam, ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam, ChatCompletionAssistantMessageParam, ChatCompletionToolMessageParam, ChatCompletionFunctionMessageParam], SkipValidation]]]], required=True),
 'answer': FieldInfo(annotation=list[str], required=True),
 'state': FieldInfo(annotation=list[dict[str, Any]], required=True),
 'info': FieldInfo(annotation=list[dict[str, Any]], required=True),
 'task': FieldInfo(annotation=list[str], required=True),
 'reward': FieldInfo(annotation=list[float], required=True),
 'metrics': FieldInfo(annotation=dict[str, list[float]